In [93]:
import copy
from functions.tools import decode_shape_binaries_str, encode_shape_binaries

def neighbors(polyomino):
    """
    ARGS: polyomino: a list of tuples representing the squares of a polyomino
    RETURN: a list of tuples representing the squares that are valid neighbors of the polyomino
    """
    found = []
    for (x,y) in polyomino:
        for delta_x, delta_y in [(1,0), (-1,0), (0,1), (0,-1)]:
            if y + delta_y < 0 or (y + delta_y == 0 and x + delta_x < 0):
                continue
            new_square = (x + delta_x, y + delta_y)
            if new_square not in found:
                found.append(new_square)
    return found

def redelmeier(n):
    """
    ARGS: n: the size of the polyominoes to count
    Caller function to redelmeier_recursion. Initializes the counts list and calls the recursive function.
    """
    counts = [0] * (n+1)
    counts[0] = 1
    polyomino = []
    untried_set = [(0,0)]
    redelmeier_recursion(n, counts, polyomino, untried_set)
    return counts

def redelmeier_recursion(n, counts, polyomino, untried_set):
    while len(untried_set) > 0:
        new_square = untried_set.pop()
        new_untried_set = copy.copy(untried_set)
        new_square_neighbors = neighbors([new_square])
        polyomino_neighbors = neighbors(polyomino)
        for s in new_square_neighbors:
            if s not in polyomino_neighbors and s not in polyomino:
                new_untried_set.append(s)
        new_polyomino = copy.copy(polyomino)
        new_polyomino.append(new_square)
        counts[len(new_polyomino)] += 1
        if len(new_polyomino) < n:
            redelmeier_recursion(n, counts, new_polyomino, new_untried_set)

print(redelmeier(10))


 

[1, 1, 2, 6, 19, 63, 216, 760, 2725, 9910, 36446]


# Modified Redelmeier Implementation

In [94]:
import copy
def neighbors(polyomino):
    """
    ARGS: polyomino: a list of tuples representing the squares of a polyomino
    RETURN: a list of tuples representing the squares that are valid neighbors of the polyomino
    """
    found = []
    for (x,y) in polyomino:
        for delta_x, delta_y in [(1,0), (-1,0), (0,1), (0,-1)]:
            if y + delta_y < 0 or (y + delta_y == 0 and x + delta_x < 0):
                continue
            new_square = (x + delta_x, y + delta_y)
            if new_square not in found:
                found.append(new_square)
    return found

def redelmeier(n):
    """
    ARGS: n: the size of the polyominoes to count
    Caller function to redelmeier_recursion. Initializes the counts list and calls the recursive function.
    """
    polyomino = []
    n_ominoes = []
    untried_set = [(0,0)]
    redelmeier_recursion(n, n_ominoes, polyomino, untried_set)
    return n_ominoes

def redelmeier_recursion(n, n_ominoes, polyomino, untried_set):
    while len(untried_set) > 0:
        new_square = untried_set.pop()
        new_untried_set = copy.copy(untried_set)
        new_square_neighbors = neighbors([new_square])
        polyomino_neighbors = neighbors(polyomino)
        for s in new_square_neighbors:
            if s not in polyomino_neighbors and s not in polyomino:
                new_untried_set.append(s)
        new_polyomino = copy.copy(polyomino)
        new_polyomino.append(new_square)
        if len(new_polyomino) < n:
            redelmeier_recursion(n, n_ominoes, new_polyomino, new_untried_set)
        else:
            n_ominoes.append(new_polyomino)


decominoes = redelmeier(10)
print(len(decominoes))


36446


In [95]:

# Converts tuple representation of polyomino to grid representation
def tuple_polyomino_to_grid(tuple_polyomino):
    min_x = min([x for (x,y) in tuple_polyomino])
    min_y = min([y for (x,y) in tuple_polyomino])
    tuple_polyomino = [(x - min_x, y - min_y) for (x,y) in tuple_polyomino]
    grid = [['0' for i in range(10)] for j in range(10)]
    for (x,y) in tuple_polyomino:
        grid[y][x] = '1'
    return grid


# Converts grid representation to string representation
def grid_to_string(grid):
    string = ""
    for row in grid:
        string += " ".join(row).replace('1','\u25a0').replace('0','_') + "\n"
    return string







 

In [96]:
# print(tuple_polyomino_to_grid(decominoes[0]))
index = 20
print(grid_to_string(tuple_polyomino_to_grid(decominoes[index])))
print(decominoes[index])


_ ■ _ _ _ _ _ _ _ _
_ ■ _ _ _ _ _ _ _ _
_ ■ _ _ _ _ _ _ _ _
_ ■ _ _ _ _ _ _ _ _
_ ■ _ _ _ _ _ _ _ _
_ ■ _ _ _ _ _ _ _ _
_ ■ _ _ _ _ _ _ _ _
■ ■ ■ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _ _ _

[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (-1, 7), (1, 7)]


# Enumerate free decominoes!

In [97]:
# For each decomino in decominoes, transform it into a set of its rotations and reflections
# If not in unique_decominoes, add it to unique_decominoes



def get_rotations(decomino):
    rotate = lambda decomino: [(y,-x) for (x,y) in decomino]
    rotations = []
    for _ in range(4):      
        decomino = rotate(decomino)
        rotations.append(decomino)
    return rotations


def get_reflections(decomino):
    return [[(-x,y) for (x,y) in decomino], [(x,-y) for (x,y) in decomino]]

# Takes in the fixed decominoes and returns the smaller set of free decominoes
def get_unique_decominoes(decominoes):
    unique_decominoes = set()
    for decomino in decominoes:
        transformations = [] # A list of tuple representations of the decomino's rotations and reflections
        for rotation in get_rotations(decomino):
            transformations.append(rotation)
            for reflection in get_reflections(rotation):
                transformations.append(reflection)
        transformations_grids = map(tuple_polyomino_to_grid, transformations)
        transformation_encodings_set = set(map(encode_shape_binaries, transformations_grids))
        if not transformation_encodings_set.intersection(unique_decominoes): # Transform every possible way and check if any of them are already in the set
            unique_decominoes.add(encode_shape_binaries(tuple_polyomino_to_grid(decomino)))
    return unique_decominoes
   
    
free_decominoes = get_unique_decominoes(decominoes)



In [98]:

print(list(free_decominoes)[2])
        

320 960 256 256 768 0 0 0 0 0
